In [3]:
import numpy as np
from sklearn.neural_network import MLPClassifier
import pandas
import time
import os
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, recall_score, f1_score, confusion_matrix, \
    precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier


In [4]:
first_number = lambda s, end=1: ((first_number(s, end + 1) if end < len(s) and s[end].isdigit() else int(s[:end])) if s[
    0].isdigit() else first_number(s[1:])) if len(s) else None

estados = {'Amapá': 26, 'Roraima': 25, 'Rondônia': 23, 'Tocantins': 19, 'Pará': 20, 'Acre': 24, 'Amazonas': 22,
           'Paraíba': 14, 'Piauí': 17, 'Pernambuco': 13, 'Bahia': 10, 'Alagoas': 12, 'Rio Grande do Norte': 15, 'Ceará': 16, 'Sergipe': 11, 'Maranhão': 18,
           'Mato Grosso do Sul': 4, 'Goiás': 6, 'Mato Grosso': 21, 'Distrito Federal': 6,
           'Minas Gerais': 7, 'Rio de Janeiro': 8, 'Espírito Santo': 9, 'São Paulo': 5,
           'Santa Catarina': 2, 'Rio Grande do Sul': 1, 'Paraná': 3}
estados.update({estados[uf]: uf for uf in estados})
months = {
    'janeiro': 1,
    'fevereiro': 2,
    'março': 3,
    'abril': 4,
    'maio': 5,
    'junho': 6,
    'julho': 7,
    'agosto': 8,
    'setembro': 9,
    'outubro': 10,
    'novembro': 11,
    'dezembro': 12
}
months.update({months[m]: m for m in months})

sex = {'Total': 0,
       'Sexo NI': 1,
       'Masculino': 2,
       'Feminino': 3}
sex.update({sex[s]: s for s in sex})

gap = lambda predicted, correct: 1 - (predicted / correct)
avg_gap = lambda predicted, correct: sum(gap(predicted[i], correct[i]) for i in range(len(correct))) / len(correct)
def sd_gap (predicted, correct):
	avg = avg_gap(predicted, correct)
	return (sum((avg - gap(predicted[i], correct[i]))**2 for i in range(len(correct)))/len(correct))**0.5

crime_convert = lambda cr, crime_list: crime_list[cr] if type(cr) == int else crime_list.index(cr)

timestamp = lambda lt=None: timestamp(time.localtime()) if lt is None else (lt[2::-1] + lt[3:6])
s_timestamp = lambda t=None: s_timestamp(timestamp()) if t is None else '%02d/%02d/%d %02d:%02d:%02d' % t

folder = './dados/'


def ibge(caminho=folder):
    ibge = {}
    raw_ibge = {}

    for arq in os.listdir(caminho):
        if arq.strip().lower().endswith('.xls'):

            ano = first_number(arq)
            print('\n' + s_timestamp() + '\t', ano, '\t', arq)

            if ano is None or ano < 1000:
                continue

            print(s_timestamp(), '\tReading file....')

            i = time.time_ns()
            raw_ibge[ano] = pandas.read_excel(pandas.ExcelFile(caminho + arq), 'BRASIL E UFs')
            f = time.time_ns()

            print(s_timestamp(), '\t', len(raw_ibge[ano]), 'raw rows in', (f - i) / 1000000, 'ms')

            ibge[ano] = {}
            uf_col = pop_col = []

            for col in set(raw_ibge[ano]):
                t = n = 0
                dados = []
                for i in range(len(raw_ibge[ano][col])):
                    if type(raw_ibge[ano][col][i]) == str:
                        t += 1
                    elif type(raw_ibge[ano][col][i]) == int:
                        n += 1
                    dados.append(raw_ibge[ano][col][i])

                if t > n:
                    print(s_timestamp(), '\tUF:', repr(col))
                    uf_col = dados
                elif n > 0 and t > 0:
                    print(s_timestamp(), '\tEstimativa:', repr(col))
                    pop_col = [(first_number(valor.strip().replace('.', '')) if type(valor) == str else valor) for valor
                               in dados]

            for i in range(len(pop_col)):
                if pop_col[i] is None or type(pop_col[i]) == float or type(uf_col[i]) == float:
                    continue

                ibge[ano][uf_col[i].strip()] = pop_col[i]

    return ibge, raw_ibge


def seg_pub(arq=folder + 'indicadoressegurancapublicauf (1).xls', ibge_pop=None, full_pop=False,
            test_time={(2021, 10), (2021, 11), (2021, 12)}):
    if ibge_pop == None:
        ibge_pop = ibge()[0]

    testes = []
    seg_pub = []	# dados do treinamento
    seg_pub_xls = pandas.ExcelFile(arq)
    ocorr = pandas.read_excel(seg_pub_xls, 'Ocorrências')
    vitim = pandas.read_excel(seg_pub_xls, 'Vítimas')

    crimes = list(set(vitim['Tipo Crime']).union(set(ocorr['Tipo Crime'])))
    crimes.sort()

    for i in range(len(ocorr)):

        crime = ocorr['Tipo Crime'][i]
        ano = ocorr['Ano'][i]
        mes = months[ocorr['Mês'][i].strip().lower()]
        uf = ocorr['UF'][i]
        ln = {}

        if ano not in ibge_pop:
            continue

        if full_pop:
            ln.update(ibge_pop[ano])
        else:
            ln['Pop'] = ibge_pop[ano][uf]

        dados = testes if (ano, mes) in test_time else seg_pub
        dados.append(ln)

        ln['UF'] = estados[uf]
        ln['Crime'] = crime_convert(crime, crimes)
        ln['Sexo'] = sex['Total']

        ln['Ano'] = ano
        ln['Mês'] = mes
        ln['Ocorrências'] = ocorr['Ocorrências'][i]

    for i in range(len(vitim)):
        crime = vitim['Tipo Crime'][i]
        sexo = vitim['Sexo da Vítima'][i]
        mes = months[vitim['Mês'][i].strip().lower()]
        ano = vitim['Ano'][i]
        uf = vitim['UF'][i]

        ln = {}

        if not ano in ibge_pop:
            continue
        if full_pop:
            ln.update(ibge_pop[ano])
        else:
            ln['Pop'] = ibge_pop[ano][uf]

        dados = testes if (ano, mes) in test_time else seg_pub
        dados.append(ln)

        ln['UF'] = estados[uf]
        ln['Crime'] = crime_convert(crime, crimes)
        ln['Sexo'] = sex[sexo]
        ln['Ano'] = ano
        ln['Mês'] = mes
        ln['Ocorrências'] = vitim['Vítimas'][i]

    return seg_pub, testes, crimes

def crimes_por_estado (dados):
    c = {}
    d = []
    for ln in dados:
        if not ln['UF'] in c:
            c[ln['UF']] = {}
        if not ln['Ano'] in c[ln['UF']]:     
            c[ln['UF']][ln['Ano']] = {}
        if not ln['Mês'] in c[ln['UF']][ln['Ano']]:     
            c[ln['UF']][ln['Ano']][ln['Mês']] = {}
        if not ln['Sexo'] in c[ln['UF']][ln['Ano']][ln['Mês']]:    
            c[ln['UF']][ln['Ano']][ln['Mês']][ln['Sexo']] = {}

        if ln['Crime'] in c[ln['UF']][ln['Ano']][ln['Mês']][ln['Sexo']]:    
            c[ln['UF']][ln['Ano']][ln['Mês']][ln['Sexo']][ln['Crime']]['Ocorrências'] += ln['Ocorrências']
        else:    
            ln = dict(ln)
            d.append(ln)
            c[ln['UF']][ln['Ano']][ln['Mês']][ln['Sexo']][ln['Crime']] = ln
            ln.pop('Crime')
    return d        

dados, testes, tipos = seg_pub()

#'''# soma todas as ocorrências de cada crime em um mesmo UF, Ano, Mês e Sexo
dados = crimes_por_estado(dados)
testes = crimes_por_estado(testes)
#'''

def resultados_numericos(corretos, preditos):
    corretos_num = []
    preditos_num = []
    for i in range(len(corretos)):
        corretos_num.append(corretos[i][-1][0])
        preditos_num.append(preditos[i][-1][0] if abs(preditos[i][-1][0] / (corretos[i][-1][0] + 10) - 1) > 0.1 else corretos[i][-1][0])
    return corretos_num, preditos_num



31/03/2023 11:54:21	 2015 	 estimativa_dou_2015_20150915.xls
31/03/2023 11:54:21 	Reading file....
31/03/2023 11:54:21 	 41 raw rows in 98.893761 ms
31/03/2023 11:54:21 	Estimativa: 'Unnamed: 2'
31/03/2023 11:54:21 	UF: 'Unnamed: 0'

31/03/2023 11:54:21	 2018 	 estimativa_dou_2018_20181019.xls
31/03/2023 11:54:21 	Reading file....
31/03/2023 11:54:21 	 40 raw rows in 63.018328 ms
31/03/2023 11:54:21 	Estimativa: 'Unnamed: 2'
31/03/2023 11:54:21 	UF: 'ESTIMATIVAS DA POPULAÇÃO RESIDENTE NO BRASIL E UNIDADES DA FEDERAÇÃO COM DATA DE REFERÊNCIA EM 1º DE JULHO DE 2018'

31/03/2023 11:54:21	 None 	 indicadoressegurancapublicauf.xls

31/03/2023 11:54:21	 2019 	 estimativa_dou_2019.xls
31/03/2023 11:54:21 	Reading file....
31/03/2023 11:54:21 	 41 raw rows in 70.941164 ms
31/03/2023 11:54:21 	Estimativa: 'Unnamed: 2'
31/03/2023 11:54:21 	UF: 'ESTIMATIVAS DA POPULAÇÃO RESIDENTE NO BRASIL E UNIDADES DA FEDERAÇÃO COM DATA DE REFERÊNCIA EM 1º DE JULHO DE 2019'

31/03/2023 11:54:21	 2021 	 estimat

In [6]:
def treinar_testar(model, data, test, types, y_cols=['Ocorrências'], normalize = False):
    print(s_timestamp(), 'Training....', model)


    correct = []
    predicted = []

    x = []
    y = []

    x_cols = [c for c in data[0] if c not in y_cols]
    max_values = {c: (max(ln[c] for ln in data) if normalize else 1) for c in data[0]}

    #print(x_cols, '\t', y_cols)
    for ln in data:
        x.append([ln[c]/max_values[c] for c in x_cols])
        y.append([ln[c]/max_values[c] for c in y_cols])

    ti = time.time_ns()
    model.fit(x,np.array(y).ravel())
    tf = time.time_ns()

    print(s_timestamp(), (tf - ti) / 1000000, 'ms')

    ti = time.time_ns()
        # Crime,Sexo,Pop,Ano,Mês -> Ocorrências
    for ln in test:
        x = [ln[c]/max_values[c] for c in x_cols]
        p = model.predict([x])
        y = [p[i]*max_values[y_cols[i]] for i in range(len(y_cols))]
        t = [ln[c]*max_values[c] for c in y_cols]

        crime = sexo = uf = ''
        if not normalize:
            '''
            crime = crime_convert(x[x_cols.index('Crime')], types)
            sexo = sex[x[x_cols.index('Sexo')]]
            uf = estados[x[x_cols.index('UF')]]
            #'''

        # [contexto], [entrada], [saída]
        correct.append(([uf, crime, sexo], x, t))
        predicted.append(([uf, crime, sexo], x, y))
    tf = time.time_ns()

    print(s_timestamp(), 'Testing for', (tf - ti) / 1000000, 'ms')

    return correct, predicted


In [7]:
corretos, preditos = treinar_testar(LinearRegression(), dados, testes, tipos)
num_corretos, num_preditos = resultados_numericos(corretos, preditos)

31/03/2023 11:54:27 Training.... LinearRegression()
31/03/2023 11:54:27 42.571723 ms
31/03/2023 11:54:27 Testing for 75.62998 ms


In [11]:
r2_score(num_corretos, num_preditos)

0.10076054005978385

In [12]:
mean_squared_error(num_corretos, num_preditos)

147448.67044274826

In [13]:
corretos, preditos = treinar_testar(KNeighborsClassifier(n_neighbors=3), dados, testes, tipos)
num_corretos, num_preditos = resultados_numericos(corretos, preditos)

31/03/2023 11:47:13 Training.... KNeighborsClassifier(n_neighbors=3)
31/03/2023 11:47:13 36.235011 ms
31/03/2023 11:47:13 Testing for 594.072593 ms


In [14]:
confusion_matrix(num_corretos, num_preditos)

array([[390,  27,   2, ...,   0,   0,   0],
       [ 86,  23,   4, ...,   0,   0,   0],
       [ 43,  10,   3, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

In [15]:
accuracy_score(num_corretos, num_preditos)

0.31165117941386705

In [16]:
precision_score(num_corretos, num_preditos, average="micro")

0.31165117941386705

In [17]:
recall_score(num_corretos, num_preditos, average="micro")

0.31165117941386705

In [18]:
f1_score(num_corretos, num_preditos, average="micro")

0.31165117941386705

In [19]:
corretos, preditos = treinar_testar(MLPClassifier(), dados, testes, tipos, normalize=False)
num_corretos, num_preditos = resultados_numericos(corretos, preditos)

31/03/2023 11:48:34 Training.... MLPClassifier()
31/03/2023 11:52:48 253555.206785 ms
31/03/2023 11:52:48 Testing for 253.009711 ms


In [27]:
confusion_matrix(num_corretos, num_preditos)

array([[457,   0,   0, ...,   0,   0,   0],
       [136,   0,   0, ...,   0,   0,   0],
       [ 78,   0,   0, ...,   0,   0,   0],
       ...,
       [  1,   0,   0, ...,   0,   0,   0],
       [  1,   0,   0, ...,   0,   0,   0],
       [  1,   0,   0, ...,   0,   0,   0]])

In [28]:
accuracy_score(num_corretos, num_preditos)

0.32666190135811296

In [29]:
precision_score(num_corretos, num_preditos, average="micro")

0.32666190135811296

In [30]:
recall_score(num_corretos, num_preditos, average="micro")

0.32666190135811296

In [31]:
f1_score(num_corretos, num_preditos, average="micro")

0.32666190135811296

In [2]:
corretos, preditos = treinar_testar(RandomForestClassifier(), dados, testes, tipos)
num_corretos, num_preditos = resultados_numericos(corretos, preditos)

NameError: name 'treinar_testar' is not defined

In [1]:
confusion_matrix(num_corretos, num_preditos)

NameError: name 'confusion_matrix' is not defined

In [ ]:
accuracy_score(num_corretos, num_preditos)

In [ ]:
precision_score(num_corretos, num_preditos, average="micro")

In [ ]:
recall_score(num_corretos, num_preditos, average="micro")

In [ ]:
f1_score(num_corretos, num_preditos, average="micro")